<center>
    <figure>
        <img src='https://i.postimg.cc/7h4NnBFg/augmented-tiles-with-25-bubbles.jpg'>
        <figcaption>This image was produced using masks and tiling methods from Jirka Borovec and custom toolkit for generating artifacts from Systems Imaging Bioinformatics Lab at the University of Michigan </figcaption>
    </figure>
</center>
<h1><center>HuBMAP + HPA - Hacking the Human Body</center></h1>

# 1. <a id='Introduction'>Introduction</a>

### 1.1. About this notebook

In this notebook I will **merely showcase** work of a team that worked on [Stress Testing Pathology Models with Generated Artifacts](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8721870/pdf/JPI-12-54.pdf) paper and [custom toolkit](https://github.com/Systems-Imaging-Bioinformatics-Lab/histopath_failure_modes) they have developed for generating artifacts on tissue imaging. **I HAVE NOT DONE ANY WORK IN THIS NOTEBOOK**. I found this paper, read it and really liked it. Also I've used the [following notebook](https://www.kaggle.com/code/jirkaborovec/ftus-segm-decompose-large-images-to-tiles/notebook) by [Jirka Borovec](https://www.kaggle.com/jirkaborovec) for spliting image into tiles. As well as dataset he made public on [image annotation masks](https://www.kaggle.com/datasets/jirkaborovec/hacking-the-human-body-annotation-masks).

**Again I will say it. This notebook does not showcase my work. It is work of [Stress Testing Pathology Models with Generated Artifacts](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8721870/pdf/JPI-12-54.pdf) paper and the people who worked hard on it.**

**I have only found it and wanted to share and showcase their work.**

### 1.2. Materials used directly
#### Notebooks:
- [FTUs⚕️Segm: decompose 🖽 large images to tiles](https://www.kaggle.com/code/jirkaborovec/ftus-segm-decompose-large-images-to-tiles/notebook) by [Jirka Borovec](https://www.kaggle.com/jirkaborovec)

#### Papers:
- [Stress Testing Pathology Models with Generated Artifacts](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8721870/pdf/JPI-12-54.pdf) by **Nicholas Chandler Wang, Jeremy Kaplan, Joonsang Lee, Jeffrey Hodgin, Aaron Udager, Arvind Rao**.

    Custom toolkit they developed can be found here: [Github repo link](https://github.com/Systems-Imaging-Bioinformatics-Lab/histopath_failure_modes) /Link in the Conclusions section under Availability headline/

In [ ]:
# Here we import/install some libraries we will be using throughout this notebook
import numpy as np
from scipy import interpolate
from scipy.ndimage import morphology
from scipy.stats import multivariate_normal

from PIL import Image, ImageDraw
from skimage import color
import cv2

import matplotlib.pyplot as plt

!pip install -q deconvolution
from deconvolution import Deconvolution

import os
import warnings
warnings.filterwarnings("ignore")

# 2. Image tiling method and mask annotation dataset by [Jirka Borovec](https://www.kaggle.com/jirkaborovec) from [FTUs⚕️Segm: decompose 🖽 large images to tiles](https://www.kaggle.com/code/jirkaborovec/ftus-segm-decompose-large-images-to-tiles/notebook) notebook

In [ ]:
def tile_image(p_img, folder, size: int = 1024) -> list:
    w = h = size
    im = np.array(Image.open(p_img))
    # https://stackoverflow.com/a/47581978/4521646
    tiles = [im[i:(i + h), j:(j + w), ...] for i in range(0, im.shape[0], h) for j in range(0, im.shape[1], w)]
    idxs = [(i, (i + h), j, (j + w)) for i in range(0, im.shape[0], h) for j in range(0, im.shape[1], w)]
    name, _ = os.path.splitext(os.path.basename(p_img))
    files = []
    for k, tile in enumerate(tiles):
        if tile.shape[:2] != (h, w):
            tile_ = tile
            tile = np.zeros_like(tiles[0])
            tile[:tile_.shape[0], :tile_.shape[1], ...] = tile_
        p_img = os.path.join(folder, f"{name}_{k:03}.png")
        Image.fromarray(tile).save(p_img)
        files.append(p_img)
    return files, idxs

DATASET_IMAGES = "/kaggle/input/hubmap-organ-segmentation/train_images/"
DATASET_MASKS = "/kaggle/input/hacking-the-human-body-annotation-masks/train_masks"

!mkdir -p /kaggle/working/temp/images
!mkdir -p /kaggle/working/temp/masks

tiles_img, _ = tile_image(os.path.join(DATASET_IMAGES, "10044.tiff"), "/kaggle/working/temp/images", size = 512)
tiles_seg, idxs = tile_image(os.path.join(DATASET_MASKS, "10044.png"), "/kaggle/working/temp/masks", size = 512)

!ls -lh /kaggle/working/temp/images
!ls -lh /kaggle/working/temp/masks

In [ ]:
nb_tiles_sqrt = int(np.sqrt(len(tiles_img)))

fig, axes = plt.subplots(nrows = nb_tiles_sqrt, ncols = nb_tiles_sqrt, figsize = (9, 9))
for i, (p_img, p_seg) in enumerate(zip(tiles_img, tiles_seg)):
    img = plt.imread(p_img)
    mask = np.array(Image.open(p_seg))
    axes[i // nb_tiles_sqrt, i % nb_tiles_sqrt].imshow(color.label2rgb(mask, img, bg_label = 0, bg_color = (1.,1.,1.), alpha = 0.25))
    axes[i // nb_tiles_sqrt, i % nb_tiles_sqrt].set_axis_off()

fig.suptitle("Original Image", fontsize = 18)
fig.tight_layout()

# 3. [Histopath Failure Modes](https://github.com/Systems-Imaging-Bioinformatics-Lab/histopath_failure_modes.git) from [Systems Imaging Bioinformatics Lab at the University of Michigan](https://sibl.lab.medicine.umich.edu)

In [ ]:
# Now we clone Histopath Failure Modes from Systems Imaging Bioinformatics Lab at the University of Michigan
!git clone https://github.com/Systems-Imaging-Bioinformatics-Lab/histopath_failure_modes.git
%cd histopath_failure_modes

### Now in their work they have studied 7 artifacts. 

- Bubbles
- Tissue fold
- Illumination
- Marker line
- Sectioning
- Stain alteration
- Tissue tears

<center>
    <figure>
        <img src='https://i.postimg.cc/VsqF5N1h/artifacts.png'>
        <figcaption>This image is displayed in Methods section of "Stress Testing Pathology Models with Generated Artifacts" work from Systems Imaging Bioinformatics Lab at the University of Michigan </figcaption>
    </figure>
</center>

In [ ]:
# Importing custom artifact generation toolkit from 
# Systems Imaging Bioinformatics Lab at the University of Michigan
from image_manipulation import img_manip

In [ ]:
# Now here let's define some functions for ease of use throughout this notebook.
def plot_tissue_with_generated_artifact(img, mask, augmented_img, augmentation):
    img = np.array(img)
    mask = np.array(mask)
    augmented_img = np.array(augmented_img)
    fig, axs = plt.subplots(1, 2, figsize = (16, 8), sharex = True, sharey = True)

    axs[0].imshow(color.label2rgb(mask, img, bg_label = 0, bg_color = (1.,1.,1.), alpha = 0.25))
    axs[0].set_title("Original image", fontsize = 14)
    axs[0].axis('off')

    axs[1].imshow(color.label2rgb(mask, augmented_img, bg_label = 0, bg_color = (1.,1.,1.), alpha = 0.25))
    axs[1].set_title(f"{augmentation}", fontsize = 14)
    axs[1].axis('off')
    plt.show();
    
# Following function is for inspecting what is happening on image when 'Illumination' is applied as 
# we can't directly observe the changes made.
def display_color_histogram(img, augmented_img, augmentation):
    augmented_img = np.array(augmented_img)
    colors = ("red", "green", "blue")
    channel_ids = (0, 1, 2)

    fig, axs = plt.subplots(1, 2, figsize = (18, 6), sharex = True, sharey = True)

    for channel_id, c in zip(channel_ids, colors):
        img = np.asarray(img)
        histogram, bin_edges = np.histogram(img[:, :, channel_id], bins = 12, range = (0, 256))
        axs[0].plot(bin_edges[0:-1], histogram, color = c, label = c)

    axs[0].spines['top'].set_visible(False)
    axs[0].spines['right'].set_visible(False)
    axs[0].set_title("Original image color histogram", fontsize = 16)

    for channel_id, c in zip(channel_ids, colors):
        img = np.asarray(img)
        histogram, bin_edges = np.histogram(augmented_img[:, :, channel_id], bins = 12, range = (0, 256))
        axs[1].plot(bin_edges[0:-1], histogram, color = c, label = c)

    axs[1].spines['top'].set_visible(False)
    axs[1].spines['right'].set_visible(False)
    axs[1].set_title(f"{augmentation}", fontsize = 16)

    fig.text(0.5, 0.04, 'Color value', ha='center', fontsize = 12)
    fig.text(0.04, 0.5, 'Pixel count', va='center', rotation='vertical', fontsize = 12);

In [ ]:
# Let's only use one tile we got from using Jirka Borovec's notebook and masks he made accessible
fig, ax = plt.subplots(figsize = (14, 8))

img = plt.imread(tiles_img[8])
mask = np.array(Image.open(tiles_seg[8]))

ax.imshow(color.label2rgb(mask, img, bg_label = 0, bg_color = (1.,1.,1.), alpha = 0.25))
ax.set_axis_off()

# 3.1. Bubbles

**In THEIR WORK they have studied that** `Air bubbles can be introduced by air getting underneath the coverslip, while nuclear bubbles can be caused by heat or other conditions that cause protein coagulation.`

In [ ]:
img = Image.open(tiles_img[8])
mask = Image.open(tiles_seg[8])
# Optional parameters for add_bubbles() method includes:
    #     random_seed: int          The random seed for numpy for consistent generation
    #     nBubbles: int (+)         The number of bubbles to generate in the image
    #     maxWidth: float (+)       The maximum width of the randomized bubbles (roughly), in pixels
    #     alpha: float (0-1)        The alpha transparency of the bubble layer (1 = opaque, 0 = transparent)
    #     edgeWidth: float (+)      The width of the darker edge of the bubble, in pixels
    #     edgeColorMult:            The RGB multiplier of the edge of the bubble 
    #        3 float vector         -Relative to the mean RGB color of the image
    #     rgbVal: 3 float vector    The RGB color of the bubbles

# Here the default value fol nBubbles is 25
img_with_bubbles = img_manip.add_bubbles(img)

plot_tissue_with_generated_artifact(img, mask, img_with_bubbles, "Image with Bubbles")

In [ ]:
# Let's play around with different nBubbles value
nBubbles = np.arange(10, 40, 5)

# Image tile
img = Image.open(tiles_img[8])

fig, axs = plt.subplots(3, 2, figsize = (20, 10))
for i, ax in enumerate(axs.flat):
    image_with_tear = np.array(img_manip.add_bubbles(img, nBubbles = nBubbles[i]))
    ax.imshow(image_with_tear)
    ax.set_title(f"Bubbles: nBubbles = {nBubbles[i]}", fontsize = 14)
    ax.axis('off')
    
fig.suptitle("Adding bubbles to image tile", fontsize = 16)
fig.tight_layout();

In [ ]:
# Let's play around with different maxWidth value
# This parameter sets maximum width of randomly generated bubbles. Default value is 50
maxWidth = np.arange(50, 70, 5)

# Image tile
img = Image.open(tiles_img[8])

fig, axs = plt.subplots(2, 2, figsize = (16, 8))
for i, ax in enumerate(axs.flat):
    image_with_tear = np.array(img_manip.add_bubbles(img, maxWidth = maxWidth[i]))
    ax.imshow(image_with_tear)
    ax.set_title(f"Bubbles: maxWidth = {maxWidth[i]}", fontsize = 14)
    ax.axis('off')
    
fig.suptitle("Adding bubbles to image tile. Playing around with maxWidth parameter", fontsize = 16)
fig.tight_layout();

# 3.2. Tissue fold

**In their work they have studied that** `Tissue folds occur when the thin tissue slice is not uniformly spread out and folds back over itself, often due to heterogeneous material properties within the sample.`[[16]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6097380/)

In [ ]:
img = Image.open(tiles_img[8])
mask = Image.open(tiles_seg[8])
# Optional parameters include:
    #     sampArr: numpy arr        A numpy array the size of the input image
    #     - used for recursion      - if the input image is not where the tissue should be sampled from
    #     sampSpl: n x 2 numpy arr  You can optionally specify the sampled spline (non-random)
    #     - used for recursion      
    #     inPts: n x 2 numpy arr    Used to prespecify the handle points of the spline
    #                               - Note: this is not random
    #     random_seed: int          The random seed for numpy for consistent generation
    #     scaleXY: 2 float vector   Used to scale the sampling bounding box, if the sample region should be resized
    #                               Defaults to no change between original and sampling
    #                               large scale = larger sample region
    #     width: float (+)          The width of the tissue fold region, in pixels
    #     sampShiftXY: 2 int vec    You can optionally specify the direction to shift the spline region
    #                               Defaults to a random direction at most half the size of the image
    #     randEdge: bool            Whether to add some randomness to the edge of the tissue fold region
    #                               Defaults to off
    #     nLayers: int (+)          Number of tissue layers to add to the image
    #                               Runs the function recursively, defaults to 2 layers
    #     nPts: int (+)             The number of random handle points in the spline
    #     endEdge: bool             Whether or not the start of the spline should be on the edge of the image
    #              int(0,1,2,3)     If endEdge is a nonnegative int, it specifies which edge the spline stops on
    #                               0 = Left, 1 = Top, 2 = Right, 3 = Bottom
    #              int(-4,-3,-2,-1) If endEdge is a negative int, it specifies which edge the spline stops on 
    #                               relative to the start
    #                               -4 = Same, -3 = End is 1 step clockwise (e.g. Bottom -> Left)
    #                               -2 = Opposite side, -1 = End is 1 step counterclockwise (e.g. Bottom -> Right)
    #                               Defaults to -2
image_with_tissue_fold = img_manip.add_fold(img, random_seed = 0)

plot_tissue_with_generated_artifact(img, mask, image_with_tissue_fold, "Image with Tissue Folds")

# 3.3. Illumination

In [ ]:
img = Image.open(tiles_img[8])
mask = Image.open(tiles_seg[8])
# Optional parameters include:
    #     random_seed: int          The random seed for numpy for consistent generation
    #     maxCov: float (+)         The maximum covariance (governs the size of the distributions)
    #     nNorms: int (+)           The number of Gaussian distributions used to build the uneven illumination
    #     scaleMin: float (<1)      The minimum for the random factor used to adjust the illumination
    #     scaleMax: float (>1)      The maximum for the random factor used to adjust the illumination
    #     minCovScale: float        The minimum on the range of sizes across the set of distributions
    #         Recommend >0 & ≤1
    #     minDiagCovScale: float    Affects the diagonal of the covariance matrix, and the minimum relative size 
    #         Recommend >0 & ≤1     of the two components compared to the scaled max covariance
    #     maxCrCovScale: float      The maximum relative cross covariance (1 = straight line, 0 = uncorrelated)
    #          Recommend >0 & ≤1    Affects the shape of the distributions, a higher number means more eccentricity

# Here default values for maxCov = 15, nNorms = 3, scaleMin = .8 and scaleMax = 1.1
image_with_illumination = img_manip.add_illumination(img)

plot_tissue_with_generated_artifact(img, mask, image_with_illumination, "Image with evenly generated illumination")

In [ ]:
# Let's observe color histogram of both images above
display_color_histogram(img, image_with_illumination, "Image with Illumination")

In [ ]:
# Now let's play around with different values for scaleMin
# Reminder that the default value is .8
scaleMin = np.arange(.2, 1, .1)

# Image tile
img = Image.open(tiles_img[8])

fig, axs = plt.subplots(4, 2, figsize = (20, 10))
for i, ax in enumerate(axs.flat):
    image_with_illumination = np.array(img_manip.add_illumination(img, scaleMin = scaleMin[i]))
    ax.imshow(image_with_illumination)
    ax.set_title("Illumination: scaleMin = %.1f" % scaleMin[i], fontsize = 14)
    ax.axis('off')
    
fig.suptitle("Adding illumination to image tile, playing around with scaleMin parameter", fontsize = 16)
fig.tight_layout();

In [ ]:
# Here we try applying different values for scaleMax
# Reminder that the default value is 1.1
scaleMax = np.arange(1.1, 2.3, .2)

# Image tile
img = Image.open(tiles_img[8])

fig, axs = plt.subplots(3, 2, figsize = (20, 10))
for i, ax in enumerate(axs.flat):
    image_with_illumination = np.array(img_manip.add_illumination(img, scaleMax = scaleMax[i]))
    ax.imshow(image_with_illumination)
    ax.set_title("Illumination: scaleMax = %.1f" % scaleMax[i], fontsize = 14)
    ax.axis('off')
    
fig.suptitle("Adding illumination to image tile, playing around with scaleMax parameter", fontsize = 16)
fig.tight_layout();

# 3.4. Marker line

**From their work. In the methods section of their paper:**

`Pathologists often use marking pens to delineate regions of interest on a histology slide, for measurement, or other uses. While a pathologist would recognize that these structures were not a feature of the tissue sample, an algorithm may not be able to accurately distinguish the two. As such, a marker artifact was generated to test algorithms for their sensitivity to this kind of change.`

In [ ]:
img = Image.open(tiles_img[8])
mask = Image.open(tiles_seg[8])
# Optional parameters include:
    #     random_seed: int          The random seed for numpy for consistent generation
    #     nPts: int                 The number of random handle points in the spline
    #     sampSpl: n x 2 numpy arr  You can optionally specify the sampled spline (non-random)
    #                               - Note: should be sampled densely enough (i.e. at least every pixel)
    #     inPts: n x 2 numpy arr    Used to prespecify the handle points of the spline
    #                               - Note: this is not random
    #     width: float (+)          The width of the marker line, in pixels
    #     alpha: float (0-1)        The alpha transparency of the marker layer (1 = opaque, 0 = transparent)
    #     rgbVal: 3 uint8 vector    The RGB color of the marker can be optionally specified
    #           >=0 <=255
    #     rgbRange: 3 x 2 uint8 arr The RGB range of the randomized color [[minR,maxR],[minG,maxG],[minB,maxB]]
    #           >=0 <=255           - Leans more blue heavy by default
image_with_marker = img_manip.add_marker(img)

plot_tissue_with_generated_artifact(img, mask, image_with_marker, "Image with marker lines")

# 3.5. Sectioning

***I'm sure I do not have to mention again at this point. But of course the following explanation is again from THEIR WORK and STUDY:***

`Sectioning artifacts are the result of unevenly cut regions of the tissue section during microtomy leading to varying thicknesses, and thus varying staining, across the slide`

In [ ]:
img = Image.open(tiles_img[8])
mask = Image.open(tiles_seg[8])
    #     width: float              The width of the sectioning region, in pixels
    #     random_seed: int          The random seed for numpy for consistent generation
    #     scaleMin: float           The minimum level of saturation allowed at random
    #                               -Note: this scales based off of the distance from the spline
    #     scaleMax: float           The maximum level of saturtation allowed at random
    #                               -Note: this scales based off of the distance from the spline
    #     randEdge: bool            Whether to add some randomness to the edge of the sectioning region
    #                               Defaults to on
    #     sampSpl: n x 2 numpy arr  You can optionally specify the sampled spline (non-random)
    #     - used for recursion      
    #     inPts: n x 2 numpy arr    Used to prespecify the handle points of the spline
    #                               - Note: this is not random
    #     nPts: int                 The number of random handle points in the spline
    #                               Defaults to 2
    #     endEdge: bool             Whether or not the start of the spline should be on the edge of the image
    #              int(0,1,2,3)     If endEdge is a nonnegative int, it specifies which edge the spline stops on
    #                               0 = Left, 1 = Top, 2 = Right, 3 = Bottom
    #              int(-4,-3,-2,-1) If endEdge is a negative int, it specifies which edge the spline stops on 
    #                               relative to the start
    #                               -4 = Same, -3 = End is 1 step clockwise (e.g. Bottom -> Left)
    #                               -2 = Opposite side, -1 = End is 1 step counterclockwise (e.g. Bottom -> Right)
    #                               Defaults to -2
image_with_sectioning = img_manip.add_sectioning(img)

plot_tissue_with_generated_artifact(img, mask, image_with_sectioning, "Image with sectioning")

In [ ]:
display_color_histogram(img, image_with_sectioning, "Image with uneven sectioning applied")

In [ ]:
# Values to play around with
# The default values for following parameters are width = 240, scaleMin = .5 and scaleMax = .8
width = np.arange(40, 260, 30)
scaleMin = np.arange(.1, .9, .1)
scaleMax = np.arange(.8, 0, -.1)

# Image tile
img = Image.open(tiles_img[8])

fig, axs = plt.subplots(4, 2, figsize = (20, 10))
for i, ax in enumerate(axs.flat):
    image_with_sectioning = np.array(img_manip.add_sectioning(img, width = width[i], scaleMin = scaleMin[i], 
                                                              scaleMax = scaleMax[i]))
    ax.imshow(image_with_sectioning)
    ax.set_title("Sectioning: w = %.1f, min = %.1f, max = %.1f" % (width[i], scaleMin[i], scaleMax[i]), 
                 fontsize = 14)
    ax.axis('off')
    
fig.suptitle("Adding sectioning image tile, playing around with width, scaleMin and scaleMax", fontsize = 16)
fig.tight_layout();

# 3.6. Stain alteration

In [ ]:
img = Image.open(tiles_img[8])
mask = Image.open(tiles_seg[8])
    #     adjFactor: 3 float vec    The adjustment factor for each of the three basis vectors 
    #                               (<1 = less stain, 1 = same, >1 = more stain)
    #                               Element 1: Eosin
    #                               Element 2: Hematoxylin
    #                               Element 3: Null (the remaining structure)
    #                               If set the change won't be random
    #     scaleMax: 3 float vector  The maximum amount of change (increase or decrease) to the stain levels
    #              (>=1)
    #     scaleMin: 3 float vector  The minimum amount of change (increase or decrease) to the stain levels
    #              (>=1)
    #     random_seed: int          The random seed for numpy for consistent generation
image_with_stain = img_manip.add_stain(img)

plot_tissue_with_generated_artifact(img, mask, image_with_stain, "Image with stain")

In [ ]:
display_color_histogram(img, image_with_stain, "Image with stain alteration applied")

# 3.7. Tissue tears

`Due to technical issues during the cutting process, for example, when hard particles are present in tissue samples or due to vibrations in the knife blade, tears can occur within the cut tissue section.`

In [ ]:
img = Image.open(tiles_img[8])
mask = Image.open(tiles_seg[8])
    #     sampSpl: n x 2 numpy arr  You can optionally specify the sampled spline (non-random)
    #     random_seed: int          The random seed for numpy for consistent generation
    #     nPts: int (+)             The number of random handle points in the spline
    #     minSpacing: float (+)     The minimum for the random spacing between tears, in pixels
    #     maxSpacing: float (+)     The maximum for the random spacing between tears, in pixels
    #     tearStartFactor:          The min for where the tear randomly starts along the spline, in percentage & 
    #       2 float vec             The max for where the tear randomly starts along the spline, in percentage
    #     tearEndFactor:            The min for where the tear randomly end along the spline, in percentage & 
    #       2 float vec             The max for where the tear randomly ends along the spline, in percentage
    #     dirMin:  float (+)        The minimum for randomized inline and perpendicular direction max distance in pixels
    #     dirMax:  float (+)        The maximum for randomized inline and perpendicular direction max distance in pixels
    #     inLineMax:  float (+)     You can optionally set the maximum size of the tear in the spline direction
    #     perpMax:  float (+)       You can optionally set the maximum size of the tear in the perpendicular direction
    #     ptRadius: float (+)       The size of each point's effect on the tear in pixels
    #     tearAlpha: float (0-1)    The alpha transparency of the tear layer (1 = opaque, 0 = transparent)
    #
    #     inLinePercs:              You can optionally set your own tear layer structure
    #       2 x n numpy float arr   The values are the percentage of distance in the in line direction that the tears take up
    #       n = number of layers    [.5,.3,.2] means most of the structure of the tear is set early
    #       rec. [[-,-,-],[+,+,+]]  and later layers fill it out
    #       rec. each row should    Making the matrix asymmetric betw. the + and -, could give a force component to the tear
    #          add up to 1 or -1    Should match the number of layers in the perpendicular side
    #                               
    #     perpPercs:                You can optionally set your own tear layer structure
    #       2 x n numpy float arr   The values are the % of distance in the perpendicular direction that the tears take up
    #       n = number of layers    [.5,.3,.2] means most of the structure of the tear is set early
    #       rec. [[-,-,-],[+,+,+]]  and later layers fill it out
    #       rec. each row should    Making the matrix asymmetric betw. the + and -, could give a sided-ness to the tear
    #          add up to 1 or -1    Should match the number of layers in the inline side
    #                               
    #     l1MinCt: int (+)          The first layer is set by number instead of density in the later layers
    #                               - Minimum # of pts in the first layer
    #     l1MaxCt: int (+)          The first layer is set by number instead of density in the later layers
    #                               - Maximum # of pts in the first layer
    #     minDensity:               The second layer and beyond are set by density instead of #
    #        n-1 int (+) vector     - Minimum density of points in the 2nd, 3rd, etc. layers
    #        n = number of layers
    #     maxDensity:               The second layer and beyond are set by density instead of #
    #        n-1 int (+) vector     - Minimum density of points in the 2nd, 3rd, etc. layers
    #     edgeAlpha: float (0-1)    The alpha transparency of the edge layer (1 = opaque, 0 = transparent)
    #     edgeColorMult:            The RGB multiplier of the edge of the tear 
    #        3 float vector         -Relative to the mean RGB color of the image
    #     rgbVal: 3 float vector    The RGB color of the tear (i.e. background)
    #     randEdge: bool            Whether to add some randomness to the edge of the tear
    #                               Defaults to on
image_with_tear = img_manip.add_tear(img)

plot_tissue_with_generated_artifact(img, mask, image_with_tear, "Image tear stain")

In [ ]:
# The default values for minSpacing = 20, maxSpacing = 40
minSpacing = np.arange(10, 70, 10)
maxSpacing = np.arange(30, 90, 10)

# Image tile
img = Image.open(tiles_img[8])

fig, axs = plt.subplots(3, 2, figsize = (20, 10))
for i, ax in enumerate(axs.flat):
    image_with_tear = np.array(img_manip.add_tear(img, minSpacing = minSpacing[i], maxSpacing = maxSpacing[i]))
    ax.imshow(image_with_tear)
    ax.set_title("Tear: minSpacing = %.1f, maxSpacing = %.1f" % (minSpacing[i], maxSpacing[i]), 
                 fontsize = 14)
    ax.axis('off')
    
fig.suptitle("Adding tear image tile, playing around with min and max Spacing", fontsize = 16)
fig.tight_layout();

In [ ]:
# The default values for dirMin = 10, dirMax = 30
dirMin = np.arange(10, 70, 10)
dirMax = np.arange(30, 90, 10)

# Image tile
img = Image.open(tiles_img[8])

fig, axs = plt.subplots(3, 2, figsize = (20, 10))
for i, ax in enumerate(axs.flat):
    image_with_tear = np.array(img_manip.add_tear(img, dirMin = dirMin[i], dirMax = dirMax[i]))
    ax.imshow(image_with_tear)
    ax.set_title("Tear: dirMin = %.1f, dirMax = %.1f" % (dirMin[i], dirMax[i]), 
                 fontsize = 14)
    ax.axis('off')
    
fig.suptitle("Adding tear to image tile, playing around with dirMin and dirMax", fontsize = 16)
fig.tight_layout();

# 4. Conclusion

Then again, I am very impressed by their work. And I greatly enjoyed reading it. So I wanted to share it. If you are interested in their work then please by all means go to their [toolkit](https://github.com/Systems-Imaging-Bioinformatics-Lab/histopath_failure_modes.git) and [research paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8721870/pdf/JPI-12-54.pdf) and read it. Also do check out [notebook](https://www.kaggle.com/code/jirkaborovec/ftus-segm-decompose-large-images-to-tiles/notebook) used for splitting an image into tiles by [Jirka Borovec](https://www.kaggle.com/jirkaborovec) and [image mask annotation dataset](https://www.kaggle.com/datasets/jirkaborovec/hacking-the-human-body-annotation-masks) he made accessible for public.

**Additional research papers and articles I have found interesting:**
- [A review of artifacts in histopathology](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6097380/)
- [Artifacts in Histological and Cytological Preparations](https://www.leicabiosystems.com/knowledge-pathway/artifacts-in-histological-and-cytological-preparations/)
- [Biopsy and Tissue Processing Artifacts in Oral Mucosal Tissues](https://www.ijhns.com/doi/pdf/10.5005/jp-journals-10001-1102)
- [Functional tissue units and their primary tissue motifs in multi-scale physiology](https://www.researchgate.net/publication/257533499_Functional_tissue_units_and_their_primary_tissue_motifs_in_multi-scale_physiology)
- [Description of soft tissue artifacts and related consequences on hindlimb kinematics during canine gait](https://peerj.com/articles/9379/)